# Filtragem dos dados do bango de dados

In [1]:
import pandas as pd
import numpy as np
import os
import datetime

# Froms
from gcpUtils.auth import getCredentials
from gcpUtils.bigQuery import pandasToBq, tableToPandas
from gcpUtils.google_storage_manager import *

cred = getCredentials("../bd/planejamento-animale-292719-296d49ccdea6.json")

### Filtragem Inicial

In [2]:
query = """
WITH base AS (
    SELECT
        FILIAL,
        DATA,
        COUNTIF(RESSUPRIR <> 0) AS produtos_com_ressuprimento
    FROM `planejamento-animale-292719.checklists_rollout.ANIMALE_checklist`
    WHERE DATA > '2025-01-15' 
      AND DATA < '2025-10-06'
    GROUP BY FILIAL, DATA
),

status_filial AS (
    SELECT
        FILIAL,
        DATA,
        CASE WHEN produtos_com_ressuprimento = 0 THEN 1 ELSE 0 END AS sem_ressuprimento
    FROM base
),

-- Detecta o início de cada período sem ressuprimento
marcacao_inicio AS (
    SELECT
        FILIAL,
        DATA,
        sem_ressuprimento,
        CAST(
            CASE
                WHEN sem_ressuprimento = 1
                 AND LAG(sem_ressuprimento, 1, 0) OVER (PARTITION BY FILIAL ORDER BY DATA) = 0
                THEN 1 ELSE 0
            END AS INT64
        ) AS inicio_periodo
    FROM status_filial
),

-- Cria IDs de grupos contínuos
grupos AS (
    SELECT
        FILIAL,
        DATA,
        sem_ressuprimento,
        SUM(inicio_periodo) OVER (
            PARTITION BY FILIAL ORDER BY DATA ROWS UNBOUNDED PRECEDING
        ) AS periodo_id
    FROM marcacao_inicio
    WHERE sem_ressuprimento = 1
),

-- Resume períodos
resumo AS (
    SELECT
        FILIAL,
        periodo_id AS ID_PERIODO,
        MIN(CAST(DATA AS DATE)) AS inicio_sem_ressuprimento,
        MAX(CAST(DATA AS DATE)) AS fim_sem_ressuprimento,
        DATE_DIFF(MAX(CAST(DATA AS DATE)), MIN(CAST(DATA AS DATE)), DAY) + 1 AS dias_sem_ressuprimento
    FROM grupos
    GROUP BY FILIAL, periodo_id
),

-- Classifica cada período e define se ainda está ativo
classificado AS (
    SELECT
        r.*,
        CASE 
            WHEN r.fim_sem_ressuprimento = (
                SELECT MAX(CAST(DATA AS DATE)) 
                FROM status_filial s WHERE s.FILIAL = r.FILIAL
            )
            THEN 'EM ANDAMENTO'
            ELSE 'FINALIZADO'
        END AS STATUS,
        CASE 
            WHEN r.dias_sem_ressuprimento > 30 THEN 'SUPERIOR A 30 DIAS'
            ELSE 'ATE 30 DIAS'
        END AS CLASSIFICACAO
    FROM resumo r
),

-- Agrupa filiais que tiveram ou não algum período > 30 dias
agrupado AS (
    SELECT
        FILIAL,
        MAX(CASE WHEN CLASSIFICACAO = 'SUPERIOR A 30 DIAS' THEN 1 ELSE 0 END) AS teve_periodo_maior_30
    FROM classificado
    GROUP BY FILIAL
)

-- Resultado final:
SELECT 
    a.FILIAL,
    CASE 
        WHEN a.teve_periodo_maior_30 = 1 THEN 'FILIAL COM PERÍODO > 30 DIAS'
        ELSE 'FILIAL SEM PERÍODO > 30 DIAS'
    END AS STATUS_FILIAL,
    c.ID_PERIODO,
    c.inicio_sem_ressuprimento,
    c.fim_sem_ressuprimento,
    c.dias_sem_ressuprimento,
    c.CLASSIFICACAO,
    c.STATUS
FROM agrupado a
LEFT JOIN classificado c
    ON a.FILIAL = c.FILIAL
ORDER BY a.FILIAL, c.ID_PERIODO;

"""

df_prod_desc = tableToPandas(query, 'planejamento-animale-292719', cred)
df_prod_desc.columns = df_prod_desc.columns.str.upper()

### Filtragem com a inclusão de vendas

In [3]:
query = """
WITH base AS (
    SELECT
        FILIAL,
        DATA,
        COUNTIF(RESSUPRIR <> 0) AS produtos_com_ressuprimento
    FROM `planejamento-animale-292719.checklists_rollout.ANIMALE_checklist`
    WHERE DATA > '2025-01-15' 
      AND DATA < '2025-10-06'
    GROUP BY FILIAL, DATA
),

status_filial AS (
    SELECT
        FILIAL,
        DATA,
        CASE WHEN produtos_com_ressuprimento = 0 THEN 1 ELSE 0 END AS sem_ressuprimento
    FROM base
),

-- Detecta o início de cada período sem ressuprimento
marcacao_inicio AS (
    SELECT
        FILIAL,
        DATA,
        sem_ressuprimento,
        CAST(
            CASE
                WHEN sem_ressuprimento = 1
                 AND LAG(sem_ressuprimento, 1, 0) OVER (PARTITION BY FILIAL ORDER BY DATA) = 0
                THEN 1 ELSE 0
            END AS INT64
        ) AS inicio_periodo
    FROM status_filial
),

-- Cria IDs de grupos contínuos
grupos AS (
    SELECT
        FILIAL,
        DATA,
        sem_ressuprimento,
        SUM(inicio_periodo) OVER (
            PARTITION BY FILIAL ORDER BY DATA ROWS UNBOUNDED PRECEDING
        ) AS periodo_id
    FROM marcacao_inicio
    WHERE sem_ressuprimento = 1
),

-- Resume períodos
resumo AS (
    SELECT
        FILIAL,
        periodo_id AS ID_PERIODO,
        MIN(CAST(DATA AS DATE)) AS inicio_sem_ressuprimento,
        MAX(CAST(DATA AS DATE)) AS fim_sem_ressuprimento,
        DATE_DIFF(MAX(CAST(DATA AS DATE)), MIN(CAST(DATA AS DATE)), DAY) + 1 AS dias_sem_ressuprimento
    FROM grupos
    GROUP BY FILIAL, periodo_id
),

-- Classifica cada período e define se ainda está ativo
classificado AS (
    SELECT
        r.*,
        CASE 
            WHEN r.fim_sem_ressuprimento = (
                SELECT MAX(CAST(DATA AS DATE)) 
                FROM status_filial s WHERE s.FILIAL = r.FILIAL
            )
            THEN 'EM ANDAMENTO'
            ELSE 'FINALIZADO'
        END AS STATUS,
        CASE 
            WHEN r.dias_sem_ressuprimento > 30 THEN 'SUPERIOR A 30 DIAS'
            ELSE 'ATE 30 DIAS'
        END AS CLASSIFICACAO
    FROM resumo r
),

-- Agrupa filiais que tiveram ou não algum período > 30 dias
agrupado AS (
    SELECT
        FILIAL,
        MAX(CASE WHEN CLASSIFICACAO = 'SUPERIOR A 30 DIAS' THEN 1 ELSE 0 END) AS teve_periodo_maior_30
    FROM classificado
    GROUP BY FILIAL
)

-- Resultado final:
SELECT 
    a.FILIAL,
    CASE 
        WHEN a.teve_periodo_maior_30 = 1 THEN 'FILIAL COM PERÍODO > 30 DIAS'
        ELSE 'FILIAL SEM PERÍODO > 30 DIAS'
    END AS STATUS_FILIAL,
    c.ID_PERIODO,
    c.inicio_sem_ressuprimento,
    c.fim_sem_ressuprimento,
    c.dias_sem_ressuprimento,
    c.CLASSIFICACAO,
    c.STATUS
FROM agrupado a
LEFT JOIN classificado c
    ON a.FILIAL = c.FILIAL
ORDER BY a.FILIAL, c.ID_PERIODO;

"""

df_prod_desc = tableToPandas(query, 'planejamento-animale-292719', cred)
df_prod_desc.columns = df_prod_desc.columns.str.upper()

In [5]:
local_path_in = os.path.join(os.getcwd(), 'data')
if not os.path.exists(local_path_in):
    os.makedirs(local_path_in)
file_path_in = os.path.join(local_path_in, 'ressuprimento_filiais.xlsx')
df_prod_desc.to_excel(file_path_in, index=False)

In [6]:
# Filtra apenas as filiais que têm classificação "SUPERIOR A 30 DIAS"
filiais_superior_30 = (
    df_prod_desc.loc[df_prod_desc["CLASSIFICACAO"] == "SUPERIOR A 30 DIAS", "FILIAL"]
    .drop_duplicates()
    .tolist()
)

# Filiais sem classificação superior a 30 dias
todas_filiais = df_prod_desc["FILIAL"].unique().tolist()
filiais_nao_superior_30 = [f for f in todas_filiais if f not in filiais_superior_30]

# Cria os dicionários
dict_filiais_superior_30 = {"SUPERIOR_30": filiais_superior_30}
dict_filiais_nao_superior_30 = {"NAO_SUPERIOR_30": filiais_nao_superior_30}

# quantidade de elementos em cada lista
qtd_filiais_superior_30 = len(filiais_superior_30)
qtd_filiais_nao_superior_30 = len(filiais_nao_superior_30)
qtd_total_filiais = len(todas_filiais)

print(dict_filiais_superior_30)

{'SUPERIOR_30': ['ANIMALE CIDADE SP CM', 'ANIMALE ECOMMERCE CM', 'ANIMALE MARINGA CM']}


In [7]:
import json

json_path = 'dados/todas_filiais.json'
dict_filiais = dict_filiais_nao_superior_30
output_path = 'dados/filiais_inferior_30.json'

def filtrar_filiais_por_dicionario(json_path, dict_filiais, output_path):
    nomes_filiais = []
    for valor in dict_filiais.values():
        if isinstance(valor, list):
            nomes_filiais.extend(valor)
        else:
            nomes_filiais.append(valor)
    nomes_filiais = [nome.upper().strip() for nome in nomes_filiais]

    with open(json_path, 'r', encoding='utf-8') as f:
        dados = json.load(f)

    # Filtra as filiais
    filtradas = [
        filial for filial in dados
        if isinstance(filial, dict)
        and filial.get("FILIAL", "").strip().upper() in nomes_filiais
    ]

    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(filtradas, f, indent=4, ensure_ascii=False)

filtrar_filiais_por_dicionario(json_path, dict_filiais, output_path)